## <font color="blue"> Data</font>

In [1]:
include("../src/JWAS.jl")
using DataFrames
using QTLDatasets
using JWAS.MT

In [2]:
phenofile = QTLDatasets.dataset("testMT","phenotype.txt")
genofile  = QTLDatasets.dataset("testMT","genotype.txt")
pedfile   = QTLDatasets.dataset("testMT","pedigree.txt");

### Genetic covariance matrix and residual covariance matrix

In [3]:
R      = [10.0 0.0
           0.0 1.0]
G      = [20.0 1.0
           1.0 2.0];

In [4]:
data=readtable(phenofile)

,Animal,BW,CW,age,sex
1,S1,100.0,10.0,8,M
2,D1,50.0,12.9,7,F
3,O1,150.0,13.0,3,M
4,O3,40.0,5.0,4,F


## <font color="blue">3. mulitiple traits with marker effects</font> 

#### set up model equations

In [5]:
model_equations = "BW = intercept + age + sex;
                   CW = intercept + age + sex";

In [6]:
model3    = MT.build_model(model_equations,R);

In [7]:
MT.set_covariate(model3,"age")

In [8]:
MT.add_markers(model3,genofile,G,separator=',',header=true);

The delimiters in file /Users/haocheng/.julia/v0.4/QTLDatasets/src/../data/testMT/genotype.txt is ,  .


In [9]:
Pi=Dict([1.0; 1.0]=>0.25,[1.0;0.0]=>0.25,[0.0,1.0]=>0.25,[0.0; 0.0]=>0.25)
C = [false true;true false]
out2 = MT.runMCMC(model3,data,Pi=Pi,chain_length=50,constraint=C,printout_frequency=10,methods="BayesC",output_marker_effects_frequency=1);

posterior means at sample: 10
Residual covariance matrix: 
[135.2224146514924 0.0
 0.0 0.4878686352225702]
Marker effects covariance matrix: 
[17.14784361737218 7.684547600676272
 7.684547600676272 5.1248163248837155]

π: 
Dict([0.0,1.0]=>0.17264068156810403,[1.0,0.0]=>0.3021274539375286,[1.0,1.0]=>0.3362292669034995,[0.0,0.0]=>0.18900259759086785)
posterior means at sample: 20
Residual covariance matrix: 
[84.51187854079498 0.0
 0.0 1.9231199526770242]
Marker effects covariance matrix: 
[29.29832073205885 5.501519456966839
 5.501519456966839 2.9814930243986764]

π: 
Dict([0.0,1.0]=>0.1803157004554784,[1.0,0.0]=>0.2513289583552475,[1.0,1.0]=>0.37047990028335065,[0.0,0.0]=>0.1978754409059234)
posterior means at sample: 30
Residual covariance matrix: 
[81.257008876543 0.0
 0.0 3.1006512747378663]
Marker effects covariance matrix: 
[30.303984048242846 5.054155557791729
 5.054155557791729 2.3756626949461315]

π: 
Dict([0.0,1.0]=>0.20973900461712433,[1.0,0.0]=>0.2191825265751662,[1.0,1.0]=>

In [10]:
using JWAS.QTL

In [11]:
file1="marker_effects_BW_2016-07-15T00:20:40.txt"
file2="marker_effects_CW_2016-07-15T00:20:40.txt";

In [12]:
a,b,c=get_additive_genetic_variances(model3,file1,file2);

In [13]:
get_breeding_values(model3,file1,file2)

2-element Array{Any,1}:
 4×3 DataFrames.DataFrame
│ Row │ ID   │ EBV      │ PEV     │
├─────┼──────┼──────────┼─────────┤
│ 1   │ "S1" │ -14.7203 │ 28.8167 │
│ 2   │ "D1" │ 5.74251  │ 34.8293 │
│ 3   │ "O1" │ 21.217   │ 48.5628 │
│ 4   │ "O3" │ -12.2392 │ 18.5602 │
 4×3 DataFrames.DataFrame
│ Row │ ID   │ EBV      │ PEV     │
├─────┼──────┼──────────┼─────────┤
│ 1   │ "S1" │ -2.94535 │ 2.39471 │
│ 2   │ "D1" │ 1.56458  │ 1.95413 │
│ 3   │ "O1" │ 3.70722  │ 4.90648 │
│ 4   │ "O3" │ -2.32645 │ 1.21073 │

In [18]:
Markdown.readme("JWAS")

# JWAS.jl

[![Build Status](https://travis-ci.org/reworkhow/JWAS.jl.svg?branch=master)](https://travis-ci.org/reworkhow/JWAS.jl)

JWAS.jl is an open-source software tool written in Julia for Bayesian multiple regression methods applied to genome-wide association studies and genomic prediction.

  * **homepage**: [QTL.rocks](http://QTL.rocks)
  * **Installation**: at the Julia REPL, `Pkg.clone("https://github.com/reworkhow/JWAS.jl.git")`
  * ~~**Documentation**: [available here](http://jwasjl.readthedocs.org/en/latest/)~~
  * **Examples**: [available here](http://nbviewer.jupyter.org/github/reworkhow/JWAS.jl/tree/master/test/)

### Structure of JWAS Module

```
JWAS.j

├──────── PedModule.jl

├──────── ST.jl
           ├── build_model
           ├── set_covariate
           ├── set_random
           ├── get_pedigree
           ├── add_markers
           ├── outputMCMCsamples
           ├── showMME
           ├── solve
           └── runMCMC

├──────── MT.jl
           ├── MT.build_model
           ├── MT.set_covariate
           ├── MT.set_random
           ├── MT.get_pedigree
           ├── MT.add_markers
           ├── MT.showMME
           ├── MT.solve
           └── MT.runMCMC

├──────── QTL.jl
```

### Troubleshooting

1. Get help about functions above through **?foo** or **@doc(foo)**
